In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform

In [2]:
fma_data = 'fma_metadata/'
#1 Gb heavy files
features = pd.read_csv(fma_data +'features.csv') # the pandas dataframe with the features and labels

C:\Anaconda\envs\ada\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,2

In [3]:
tracks = pd.read_csv(fma_data +'tracks.csv')
genres = pd.read_csv(fma_data +'genres.csv')
echonest = pd.read_csv(fma_data + "echonest.csv")

C:\Anaconda\envs\ada\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Anaconda\envs\ada\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,11,13,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,1

In [4]:
t = tracks.copy()
t.columns = t.iloc[0]
t.drop(t.index[0], inplace=True)
t.drop(t.index[0], inplace=True)
t = t.rename(columns={np.NaN: 'track_id'})
t = t.set_index('track_id')
t = t[t['genres'] != '[]']
t = t[['title', 'genres', 'genres_all', 'genre_top', 'id', 'name',]]
pd.io.parsers.ParserBase({'title':t.columns})._maybe_dedup_names(t.columns)
t = t.rename(columns={ t.columns[0]: 'album_title', t.columns[1] : 'track_title', t.columns[5] : 'album_id', t.columns[6] : 'artist_id'})
#rh = t.loc[(t['genre_top'] != np.nan)] | (t['genre_top'] == 'Rock') ]
t.dropna(subset=['genre_top'], inplace=True)
t.count()

0
track_title    49289
track_title    49597
genres         49598
genres_all     49598
genre_top      49598
artist_id      49598
artist_id      49598
name           49598
dtype: int64

**Exploration**

In [5]:
t.head(5)

,track_title,track_title,genres,genres_all,genre_top,artist_id,artist_id,name
track_id,,,,,,,,
2,AWOL - A Way Of Life,Food,[21],[21],Hip-Hop,1,1,AWOL
3,AWOL - A Way Of Life,Electric Ave,[21],[21],Hip-Hop,1,1,AWOL
5,AWOL - A Way Of Life,This World,[21],[21],Hip-Hop,1,1,AWOL
10,Constant Hitmaker,Freeway,[10],[10],Pop,6,6,Kurt Vile
134,AWOL - A Way Of Life,Street Music,[21],[21],Hip-Hop,1,1,AWOL


In [6]:
t['genre_top'].value_counts()

Rock                   14182
Experimental           10608
Electronic              9372
Hip-Hop                 3552
Folk                    2803
Pop                     2332
Instrumental            2079
International           1389
Classical               1230
Jazz                     571
Old-Time / Historic      554
Spoken                   423
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: genre_top, dtype: int64

**Filter the 4 Genres of interest**

In [7]:
rh = t.loc[(t['genre_top'] == 'Rock') |  (t['genre_top'] == 'Electronic') | (t['genre_top'] == 'Experimental') |\
      (t['genre_top'] == 'Hip-Hop')]

In [8]:
rh['genre_top'].value_counts()

Rock            14182
Experimental    10608
Electronic       9372
Hip-Hop          3552
Name: genre_top, dtype: int64

### Features

In [9]:
f = features.iloc[3:,1:].copy()
features_id_list = features.iloc[3:,:]['feature'].rename('track_id')
len(features_id_list), len(f)

(106574, 106574)

In [10]:
f = f.apply(lambda x : pd.to_numeric(x))
f.set_index(features_id_list, inplace=True)
f.head(5)

,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,chroma_cens.9,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
track_id,,,,,,,,,,,,,,,,,,,,,
2,7.180653,5.230309,0.249321,1.347620,1.482478,0.531371,1.481593,2.691455,0.866868,1.341231,...,0.054125,0.012226,0.012111,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448
3,1.888963,0.760539,0.345297,2.295201,1.654031,0.067592,1.366848,1.054094,0.108103,0.619185,...,0.063831,0.014212,0.017740,2.824694,0.466309,0.084578,0.063965,0.000000,1.716724,0.069330
5,0.527563,-0.077654,-0.279610,0.685883,1.937570,0.880839,-0.923192,-0.927232,0.666617,1.038546,...,0.040730,0.012691,0.014759,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861
10,3.702245,-0.291193,2.196742,-0.234449,1.367364,0.998411,1.770694,1.604566,0.521217,1.982386,...,0.074358,0.017952,0.013921,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800
20,-0.193837,-0.198527,0.201546,0.258556,0.775204,0.084794,-0.289294,-0.816410,0.043851,-0.804761,...,0.095003,0.022492,0.021355,16.669037,0.469727,0.047225,0.040039,0.000977,3.189831,0.030993


**Merge Features and tracks**

In [11]:
features_and_tracks = pd.merge(f, rh, left_index=True, right_index=True)
features_and_tracks.head(5)

,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,chroma_cens.9,...,zcr.5,zcr.6,track_title,track_title,genres,genres_all,genre_top,artist_id,artist_id,name
track_id,,,,,,,,,,,,,,,,,,,,,
2,7.180653,5.230309,0.249321,1.347620,1.482478,0.531371,1.481593,2.691455,0.866868,1.341231,...,2.089872,0.061448,AWOL - A Way Of Life,Food,[21],[21],Hip-Hop,1,1,AWOL
3,1.888963,0.760539,0.345297,2.295201,1.654031,0.067592,1.366848,1.054094,0.108103,0.619185,...,1.716724,0.069330,AWOL - A Way Of Life,Electric Ave,[21],[21],Hip-Hop,1,1,AWOL
5,0.527563,-0.077654,-0.279610,0.685883,1.937570,0.880839,-0.923192,-0.927232,0.666617,1.038546,...,2.193303,0.044861,AWOL - A Way Of Life,This World,[21],[21],Hip-Hop,1,1,AWOL
134,0.918445,0.674147,0.577818,1.281117,0.933746,0.078177,1.199204,-0.175223,0.925482,1.438509,...,1.806106,0.054623,AWOL - A Way Of Life,Street Music,[21],[21],Hip-Hop,1,1,AWOL
135,0.978402,0.623572,1.129261,-1.043970,-1.127288,-0.032642,-0.878434,-0.916234,-0.743257,0.609235,...,0.297873,0.024899,mp3,Father's Day,"[45, 58]","[58, 12, 45]",Rock,58,52,Abominog


In [12]:
np.shape(features_and_tracks)

(30909, 526)

**Create the dataframe file for later use**

In [13]:
features_and_tracks.to_pickle('data_4')